In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import urllib.request
import time
import re
import random

In [2]:
counter = 0
maxCount = 1000

In [3]:
def saveContent(url,file):
    response = requests.get(url)
    html = BeautifulSoup(response.text, "html.parser")
    title = html.select("#firstHeading")[0].text
    paragraphs = html.select("p")

    text = '\n'.join([para.text for para in paragraphs])

    textFile = open(file, "w", encoding="utf-8")
    textFile.write(text)
    textFile.close()

    global counter
    counter += 1

In [4]:
def website(url,file):
    print(url)
    response = requests.get(url)
    html = BeautifulSoup(response.text, "html.parser")
    
    paras = html.findAll('p') 
    links = []
    nextWebsites = []
    for i in range(0,len(paras)):
        links = paras[i].findAll('a') 
        if(len(links) > 0):
            for l in links:
                if not l.has_attr('href'):
                    break
                l = l['href']
                if(re.match(r"/wiki/\w*$",l)):
                    nextUrl = "https://simple.wikipedia.org" + l
                    nextFile = "." + l + ".txt"
                    nextWebsites.append((nextUrl,nextFile))
                    saveContent(nextUrl,nextFile)
                    if(counter == maxCount):
                        return True
    
    if(len(nextWebsites) == 0):
        print("the end.")
        return False
    
    while(True):
        url,file = nextWebsites[random.randint(1, len(nextWebsites))-1]
        if(website(url,file)):
            return True


In [5]:
url = "https://simple.wikipedia.org/wiki/Satellite_(artificial)" #starting website
global counter
counter = 0
website(url,"./wiki/Satellite_(artificial).txt")

In [7]:
#usuniecie pustych (błędy, błędy, błędy)
files = os.listdir('./wiki/')
for i in range(len(files)):
    fileName = './wiki/' + files[i]
    f = open(fileName,"r", encoding="utf-8")
    if(len(f.read()) == 0):
        os.remove(fileName)
    f.close()